In [1]:
import findspark
findspark.init()
findspark.find()

'/home/henry/spark'

In [2]:
import pyspark
from pyspark.sql import SparkSession, functions as F

In [3]:
spark = SparkSession.builder.appName("DNA").getOrCreate()

22/10/17 22:43:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. Load data

In [4]:
# view data
!cat data/sample.fasta

>seq1
cGTAaccaataaaaaaacaagcttaacctaattc
>seq2
agcttagTTTGGatctggccgggg
>seq3
gcggatttactcCCCCCAAAAANNaggggagagcccagataaatggagtctgtgcgtccaca
gaattcgcacca
AATAAAACCTCACCCAT
agagcccagaatttactcCCC
>seq4
gcggatttactcaggggagagcccagGGataaatggagtctgtgcgtccaca
gaattcgcacca


In [5]:
records_path = "data/sample.fasta"
# call sparkContext.textFile to create rdd from file
rdd_records = spark.sparkContext.textFile(records_path)

In [6]:
rdd_records.collect()

['>seq1',
 'cGTAaccaataaaaaaacaagcttaacctaattc',
 '>seq2',
 'agcttagTTTGGatctggccgggg',
 '>seq3',
 'gcggatttactcCCCCCAAAAANNaggggagagcccagataaatggagtctgtgcgtccaca',
 'gaattcgcacca',
 'AATAAAACCTCACCCAT',
 'agagcccagaatttactcCCC',
 '>seq4',
 'gcggatttactcaggggagagcccagGGataaatggagtctgtgcgtccaca',
 'gaattcgcacca']

## 2. DNA Base Count

### 2.1 Solution 1: User-defined function + reduce()

In [16]:
def process_records(line:str):
    '''
    Return list of tuple of (char, 1) with char='z' is number of sequence 
    and others are DNA.
    All chars are lowercase
    '''
    key_value_list = []
    if line.startswith(">"):
        key_value_list.append(("z", 1))
    else:
        chars = line.lower()
        for char in chars:
            key_value_list.append((char, 1))
    return key_value_list

In [8]:
rdd_bases = rdd_records.flatMap(process_records)

In [18]:
rdd_bases.take(10) # take 3 sample from rdd

[('z', 1),
 ('c', 1),
 ('g', 1),
 ('t', 1),
 ('a', 1),
 ('a', 1),
 ('c', 1),
 ('c', 1),
 ('a', 1),
 ('a', 1)]

In [19]:
rdd_frequencies = rdd_bases.reduceByKey(lambda x, y: x+y)
rdd_frequencies.collect()

[('c', 61), ('g', 53), ('z', 4), ('t', 45), ('a', 73), ('n', 2)]

In [14]:
rdd_frequencies.collectAsMap() # return as dict type

{'c': 61, 'g': 53, 'z': 4, 't': 45, 'a': 73, 'n': 2}

In [15]:
# using groupBy
grouped_rdd = rdd_bases.groupByKey()
frequencies_rdd = grouped_rdd.mapValues(lambda values : sum(values))
frequencies_rdd.collect()

[('c', 61), ('g', 53), ('z', 4), ('t', 45), ('a', 73), ('n', 2)]

1. Pros
- easy to understand
- no scalability issue
2. Cons
- cause memory problems if have too many (key, value)
- high load on the network and prolong the shuffle time -> bottleneck

### 2.2 Solution 2: Using HashMap (aka Dict) for processing function

In [31]:
def process_records_hashmap(line:str):
    '''
    Return list of tuple of (char, 1) with char='z' is number of sequence 
    and others are DNA.
    All chars are lowercase
    '''
    dict_dna = {}
    if line.startswith(">"):
        dict_dna["z"] = 1
    else:
        chars = line.lower()
        for char in chars:
            if char in dict_dna:
                dict_dna[char] += 1
            else:
                dict_dna[char] = 1
    key_value_list = [(k, v) for k, v in dict_dna.items()]
    return key_value_list

In [35]:
dna_line_count = rdd_records.flatMap(process_records_hashmap)
dna_line_count.take(5)

[('z', 1), ('c', 8), ('g', 2), ('t', 7), ('a', 17)]

In [37]:
dna_count = dna_line_count.reduceByKey(lambda x, y: x + y)
dna_count.collect()

[('c', 61), ('g', 53), ('z', 4), ('t', 45), ('a', 73), ('n', 2)]

1. Pros:
- Reduce number (key, value)
2. Cons
- still emitting up to six (key, value) pairs per DNA string line.

### 2.3 Solution 3: using mapPartions() transformation

In [41]:
help(pyspark.RDD.mapPartitions)

Help on function mapPartitions in module pyspark.rdd:

mapPartitions(self, f, preservesPartitioning=False)
    Return a new RDD by applying a function to each partition of this RDD.
    
    >>> rdd = sc.parallelize([1, 2, 3, 4], 2)
    >>> def f(iterator): yield sum(iterator)
    >>> rdd.mapPartitions(f).collect()
    [3, 7]



- map() is a 1-to-1 transformation
- mapPartition() is a many-to-1 transformation by applying func() to each partition.
- mapPartitions(func) transformation runs separately on each partition (block) of the RDD
- func() must be of type iterator

In [43]:
# example
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
numOfPartitions = 3
rdd_sample = spark.sparkContext.parallelize(numbers, numOfPartitions)
rdd_sample.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [44]:
rdd_sample.getNumPartitions()

3

In [45]:
def scan(iterator):
    print(list(iterator))
rdd_sample.foreachPartition(scan)

[1, 2, 3]
[4, 5, 6]
[7, 8, 9, 10]


In [47]:
def adder(iterator):
    yield sum(iterator)
    
rdd_sample.mapPartitions(adder).collect()

[6, 15, 34]

In [59]:
def process_records_partition(iterator):
    '''
    Return list of tuple of (char, 1) with char='z' is number of sequence 
    and others are DNA for each partition
    All chars are lowercase
    '''
    dict_dna = {"z": 0}
    for line in iterator:
        if line.startswith(">"):
            dict_dna["z"] += 1
        else:
            chars = line.lower()
            for char in chars:
                if char in dict_dna:
                    dict_dna[char] += 1
                else:
                    dict_dna[char] = 1
    key_value_list = [(k, v) for k, v in dict_dna.items()]
    return key_value_list

In [53]:
rdd_records.getNumPartitions()

2

In [57]:
dna_partition_count = rdd_records.mapPartitions(process_records_partition)
dna_partition_count.collect() # only two 'z' keys

[('z', 3),
 ('c', 28),
 ('g', 28),
 ('t', 24),
 ('a', 38),
 ('n', 2),
 ('z', 1),
 ('g', 25),
 ('a', 35),
 ('t', 21),
 ('c', 33)]

In [58]:
frequencies_rdd = dna_partition_count.reduceByKey(lambda a, b: a + b)
frequencies_rdd.collect()

[('c', 61), ('g', 53), ('z', 4), ('t', 45), ('a', 73), ('n', 2)]

1. Pros
- not be a threat to scalability and will not use too much memory.
2. Cons
- require custom code

### 3. Summary

- work on 'real' bigdata to test performance
- use reduceByKey() instead of groupByKey()
- use mapPartitions for reducing time